In [1]:
import numpy as np

rewards = [-1, -2, -2, 10, 1, 0]
def compute_return(start_index, chain, gamma) :
    G = 0
    # 注意，必须从后往前计算
    for state in reversed(chain) :
        r = rewards[state - 1]
        G = r + gamma * G
    return G

chain = [1, 2, 3, 6]
start_index = 0
gamma = 0.5
G = compute_return(start_index, chain, gamma)
print(G)

-2.5


矩阵形式贝尔曼方程：  
$v = r + \gamma P v$  
解：  
$v = (I - \gamma P)^{-1}r$

In [18]:
def compute(gamma, P, rewards, state_num) :
    I = np.eye(state_num)
    rewards = np.array(rewards).reshape((-1, 1))
    state_value = np.dot(np.linalg.inv(I - gamma * P), rewards)
    return state_value

P = [
    [0.9, 0.1, 0.0, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.6, 0.0, 0.4],
    [0.0, 0.0, 0.0, 0.0, 0.3, 0.7],
    [0.0, 0.2, 0.3, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
]
P = np.array(P)
# 就算不reshape, numpy也会拿行向量当列向量计算。
state_value = compute(gamma, P, rewards, 6)
print(state_value)

[[-2.01950168]
 [-2.21451846]
 [ 1.16142785]
 [10.53809283]
 [ 3.58728554]
 [ 0.        ]]


只看公式，不动手写写，总觉得不得劲

### Markov Process:  
$(\mathcal{S},\mathcal{P})$  

### Markov Reward Process:  
$(\mathcal{S},\mathcal{P},\mathcal{R},\gamma)$  

#### t时刻开始的回报:  
$G_t=R_t + \gamma G_{t+1}$  
#### 价值函数:  
##### 期望形式:  
$
\begin{align}
V(s)&=\mathbb{E}[G_t|S_t = s]\\
&=\mathbb{E}[R_t+\gamma G_{t+1}|S_t=s]\\
&=\mathbb{E}[R_t|S_t=s] + \gamma \mathbb{E}[G_{t+1}|S_t=s]
\end{align}
$  
$\mathbb{E}[G_{t+1}|S_t=s]$和$\mathbb{E}[G_{t+1}|S_t=s,S_{t+1}=s']$还是有区别的。理论上$G_{t+1}$和$s'$有关，因此前者要展开成后者。根据全期望公式，$\mathbb{E}[X]=\mathbb{E}[\mathbb{E}[X|Y]]$  
$\mathbb{E}[G_{t+1}|S_t=s]=\sum_{s'\in\mathcal{S}}p(s'|s)\mathbb{E}[G_{t+1}|S_t=s,S_{t+1}=s']$  
对应全期望公式里，$X=\{G_{t+1}|S_t=s\}$,$X|Y=\{G_{t+1}|S_t=s,S_{t+1}=s'\}$,$Y=\{S_{t+1}=s\}$  
由于马尔科夫性，$G_{t+1}$和$S_{t}=s$无关，因此$\mathbb{E}[G_{t+1}|S_t=s,S_{t+1}=s']=\mathbb{E}[G_{t+1}|S_{t+1}=s']$  
$\mathbb{E}[G_{t+1}|S_t=s]=\sum_{s'\in\mathcal{S}}p(s'|s) \mathbb{E}[G_{t+1}|S_{t+1}=s']=\sum_{s'\in\mathcal{S}}p(s'|s) V(s') = \mathbb{E}[V(s')|S_t=s]$  
最终：  
$V(s) = \mathbb{E}[R_t|S_t=s] + \gamma \mathbb{E}[V(s')|S_t=s]=\mathbb{E}[R_t + \gamma V(s')|S_t=s]$

##### 期望展开形式:
$V(s)=R_t(s) + \gamma \sum_{s' \in \mathcal{S}}p(s'|s)V(s')$  

#### 矩阵形式:  
$V = r+\gamma PV$  
$V = (I-\gamma P)^{-1} r$

### Markov Decision Process
$(\mathcal{S},\mathcal{A},\mathcal{R},\mathcal{P},\gamma)$  

#### 状态价值
$V_{\pi} = \mathbb{E}[G_t|S_t=s]$

#### 行动价值
$Q_{\pi} = \mathbb{E}[G_t|S_t=s,A_t=a]$

#### 状态价值和行动价值关系
$V_{\pi} = \sum_{a\in\mathcal{A}}\pi(s,a)Q_{\pi}(s,a)$  
$Q_{\pi} = r_t(s,a) + \gamma \sum_{s' \in\mathcal{S}}p(s'|s,a)V_{\pi}(s')$

#### 贝尔曼方程展开
$
\begin{align}
V_{\pi}(s) &= \mathbb{E}[R_t + \gamma V_{\pi}(s')|S_t=s]\\
&= \sum_{a\in\mathcal{A}}\pi(a|s)[r(s,a)+\gamma \sum_{s'\in\mathcal{S}}p(s'|s,a)V_{\pi}(s')]
\end{align}
$

$
\begin{align}
Q_{\pi}(s,a) &= r_t(s,a) + \gamma \mathbb{E}[V_{\pi}(s')|S_t=s] \\
&= r_t(s,a) + \gamma \sum_{s'\in\mathcal{S}}p(s'|s,a)\sum_{a'\in\mathcal{A}}\pi(a'|s')Q_{\pi}(s',a')
\end{align}
$

In [3]:
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态集合
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作集合
# 状态转移函数
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}
# 奖励函数
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}
gamma = 0.5  # 折扣因子
MDP = (S, A, P, R, gamma)

# 策略1,随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}
# 策略2
Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}


# 把输入的两个字符串通过“-”连接,便于使用上述定义的P、R变量
def join(str1, str2):
    return str1 + '-' + str2

In [19]:
gamma = 0.5
# 转化后的MRP的状态转移矩阵
P_from_mdp_to_mrp = [
    [0.5, 0.5, 0.0, 0.0, 0.0],
    [0.5, 0.0, 0.5, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.5, 0.5],
    [0.0, 0.1, 0.2, 0.2, 0.5],
    [0.0, 0.0, 0.0, 0.0, 1.0],
]
P_from_mdp_to_mrp = np.array(P_from_mdp_to_mrp)
R_from_mdp_to_mrp = [-0.5, -1.5, -1.0, 5.5, 0]

V = compute(gamma, P_from_mdp_to_mrp, R_from_mdp_to_mrp, 5)
print("MDP中每个状态价值分别为\n", V)

MDP中每个状态价值分别为
 [[-1.22555411]
 [-1.67666232]
 [ 0.51890482]
 [ 6.0756193 ]
 [ 0.        ]]


MDP是怎么转成MRP的？  
我原以为是策略矩阵和状态转移矩阵直接乘就可以了。但实际看了维度，对不上。  
策略$\pi(a|s)$的shape:(state_num, action_num)
状态转移概率$`p(s'|s,a)`$的shape:(state_num, action_num, state_num)
对于某个状态，从策略里切出对应的状态，shape是(action_num, 1)，转置成向量，shape(1, action_num)  
从状态转移概率里切出对应的状态，shape是(action_num, state_num)  
两者相乘，(1, action_num) 乘 (action_num, state_num) 等于(1, state_num)  
对于每个状态，都来一遍，所以最终是(state_num, state_num)，也就是状态转移概率。

In [20]:
def sample(MDP, Pi, timestep_max, number):
    ''' 采样函数,策略Pi,限制最长时间步timestep_max,总共采样序列数number '''
    S, A, P, R, gamma = MDP
    episodes = []
    for _ in range(number):
        episode = []
        timestep = 0
        s = S[np.random.randint(4)]  # 随机选择一个除s5以外的状态s作为起点
        # 当前状态为终止状态或者时间步太长时,一次采样结束
        while s != "s5" and timestep <= timestep_max:
            timestep += 1
            rand, temp = np.random.rand(), 0
            # 在状态s下根据策略选择动作
            for a_opt in A:
                temp += Pi.get(join(s, a_opt), 0)
                if temp > rand:
                    a = a_opt
                    r = R.get(join(s, a), 0)
                    break
            rand, temp = np.random.rand(), 0
            # 根据状态转移概率得到下一个状态s_next
            for s_opt in S:
                temp += P.get(join(join(s, a), s_opt), 0)
                if temp > rand:
                    s_next = s_opt
                    break
            episode.append((s, a, r, s_next))  # 把（s,a,r,s_next）元组放入序列中
            s = s_next  # s_next变成当前状态,开始接下来的循环
        episodes.append(episode)
    return episodes


# 采样5次,每个序列最长不超过20步
episodes = sample(MDP, Pi_1, 20, 5)
print('第一条序列\n', episodes[0])
print('第二条序列\n', episodes[1])
print('第五条序列\n', episodes[4])

第一条序列
 [('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s5', 0, 's5')]
第二条序列
 [('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '保持s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s1', -1, 's1'), ('s1', '前往s2', 0, 's2'), ('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's4'), ('s4', '概率前往', 1, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '概率前往', 1, 's3'), ('s3', '前往s4', -2, 's4')]
第五条序列
 [('s2', '前往s3', -2, 's3'), ('s3', '前往s4', -2, 's4'), ('s4', '前往s5', 10, 's5')]

In [21]:
# 对所有采样序列计算所有状态的价值
def MC(episodes, V, N, gamma):
    for episode in episodes:
        G = 0
        for i in range(len(episode) - 1, -1, -1):  #一个序列从后往前计算
            (s, a, r, s_next) = episode[i]
            G = r + gamma * G
            N[s] = N[s] + 1
            V[s] = V[s] + (G - V[s]) / N[s]


timestep_max = 20
# 采样1000次,可以自行修改
episodes = sample(MDP, Pi_1, timestep_max, 1000)
gamma = 0.5
V = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
N = {"s1": 0, "s2": 0, "s3": 0, "s4": 0, "s5": 0}
MC(episodes, V, N, gamma)
print("使用蒙特卡洛方法计算MDP的状态价值为\n", V)

使用蒙特卡洛方法计算MDP的状态价值为
 {'s1': -1.234926404480339, 's2': -1.6920408505160804, 's3': 0.4596046559950893, 's4': 5.910741298902233, 's5': 0}


In [40]:
def occupancy(episodes, s, a, timestep_max, gamma):
    ''' 计算状态动作对（s,a）出现的频率,以此来估算策略的占用度量 '''
    '''
     episodes里有多条轨迹，每条轨迹长度不同，理论上total_times的长度是最长轨迹的长度，为了省略，直接写1000
     所以，这个total_count，第i个元素，就是"有多少回合走到了这个时间步",理论上每个回合肯定有第一个时间步，
     所以total_times[0] == len(episodes)
     occur_times[i]是指定的(s,a)对，在时刻i出现过，就+1.
     所以occur_times[i] / total_times[i]的意思就是，有total_times[i]个回合进行到了时间步i，
     其中指定(s,a)对在i时刻出现occur_times[i]次.也就是所谓的,(s,a)在某时刻出现的概率(估计)。
    '''
    rho = 0
    total_times = np.zeros(timestep_max)  # 记录每个时间步t各被经历过几次
    occur_times = np.zeros(timestep_max)  # 记录(s_t,a_t)=(s,a)的次数
    for episode in episodes:
        for i in range(len(episode)):
            (s_opt, a_opt, r, s_next) = episode[i]
            total_times[i] += 1
            if s == s_opt and a == a_opt:
                occur_times[i] += 1
    for i in reversed(range(timestep_max)):
        if total_times[i]:
            rho += gamma**i * occur_times[i] / total_times[i]
    return (1 - gamma) * rho

gamma = 0.5
timestep_max = 1000

np.random.seed(1)
episodes_1 = sample(MDP, Pi_1, timestep_max, 1000)
episodes_2 = sample(MDP, Pi_2, timestep_max, 1000)
rho_1 = occupancy(episodes_1, "s4", "概率前往", timestep_max, gamma)
rho_2 = occupancy(episodes_2, "s4", "概率前往", timestep_max, gamma)
print(rho_1, rho_2)

0.11686852128509743 0.23391363418625283


#### 状态访问分布 state visitation distribution
吐槽一下，这本书写的太跳跃了，前后衔接很差。突然提出这个概念，不知所云，使人抓狂。  
状态访问分布$\nu^{\pi}(s) = (1-\gamma)\sum_{t=0}^{\infty} \gamma^t P^{\pi}_t(s)$  
首先，状态访问分布和占用度量，都是人为定义出来的值、构造出来的公式，类似状态价值和行动价值，是为了某种目的，而构造出来的。和稳定分布不太一样，稳定分布是对实际概率的推演，即固定策略和状态转移概率，长期运行后状态访问会稳定在一个分布上。   
为什么要有状态访问分布这个定义呢？  
因为，调整策略，其实就是在调整状态访问分布；而一个状态的状态价值，也和该状态访问之后的状态访问分布有关，为什么，因为不同状态的状态价值不同嘛，当前状态的状态价值又是未来访问状态价值的期望，期望不止有值，还有分布，可不分布影响了当前状态价值的值。  
或者说，状态价值定义在期望之上，而期望由两部分组成，概率和值。而值其实又和未来的概率和值有关，是收敛得来的。所以最终，状态价值其实取决于概率和奖励，奖励才是真正输入进这个系统的信号。而策略，调整的是访问各状态获取奖励的分布。状态价值，是在这两者作用之下得到的收敛的值。  
基于以上罗里吧嗦的一堆，可以认识到，状态访问概率，是挺重要的一个指标。所以，为了衡量这个指标，发明了以上公式，来描述长期的状态访问概率。它不是一个真实存在的值，再说一遍，但是它能衡量出长期的状态访问分布。  


#### 状态访问分布对应的贝尔曼方程
$\nu^{\pi}(s') = (1-\gamma)v_0(s') + \gamma \sum_{s\in\mathcal{S}}\sum_{a\in\mathcal{A}}\pi(a|s)p(s'|s,a) \nu^{\pi}(s)$

推导  
$\nu^{\pi}(s') = (1-\gamma)\sum_{t=0}^{\infty} \gamma^t P^{\pi}_t(s')$ 

$\nu^{\pi}(s') = (1-\gamma) (\gamma ^0P_0^{\pi}(s') + \sum_{t=1}^{\infty} \gamma^t P_t^{\pi}(s')$

$\nu^{\pi}(s') = (1-\gamma)\nu_0(s') + (1-\gamma)\sum_{t=1}^{\infty} \gamma^t P_t^{\pi}(s')$

这么一行一步比较舒服，能看到上一行，不然全在一个cell里不太好看前面写过的公式。  
关注后半部分，按状态和动作展开：

$(1-\gamma)\sum_{t=1}^{\infty} \gamma^t P_t^{\pi}(S_t=s') = (1-\gamma)\sum_{t=1}^{\infty} \sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \gamma^t \pi(a|s)p(s'|s,a)P(S_{t-1}=s'|\pi)$

$= (1-\gamma)\sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \pi(a|s)p(s'|s,a)\sum_{t=1}^{\infty} \gamma^t P(S_{t-1}=s'|\pi)$

令$t'=t-1$, 则$t=t'+1$,$S_{t'}=S_{t-1}=s$代入

$= (1-\gamma)\sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \pi(a|s)p(s'|s,a)\sum_{t'=0}^{\infty} \gamma^{t'+1} P(S_{t'}=s|\pi)$

$= (1-\gamma)\sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \pi(a|s)p(s'|s,a)\gamma \sum_{t'=0}^{\infty} \gamma^{t'} P(S_{t'}=s|\pi)$

按照定义：  
$\nu^{\pi}(s) = (1-\gamma)\sum_{t=0}^{\infty} \gamma^t P^{\pi}_t(s)$   
$\frac{\nu^{\pi}(s)}{(1-\gamma)} = \sum_{t=0}^{\infty} \gamma^t P^{\pi}_t(s)$

$= (1-\gamma)\sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \pi(a|s)p(s'|s,a)\gamma \frac{1}{1-\gamma} \nu^{\pi}(s)$

$= \gamma\sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \pi(a|s)p(s'|s,a) \nu^{\pi}(s)$

和前半部分合并代入

$\nu^{\pi}(s') = (1-\gamma)\nu_0(s') + \gamma\sum_{s\in\mathcal{S}} \sum_{a\in\mathcal{A}} \pi(a|s)p(s'|s,a) \nu^{\pi}(s)$

得证

然后是所谓的占用度量。如果状态访问分布对应状态价值，占用度量就对应行动价值。  
$\rho_{\pi}(s,a)=(1-\gamma)\sum_{t=0}^{\infty} \gamma^t P_t^{\pi}(s)\pi(a|s)$  
$= (1-\gamma)\nu_{\pi}(s) \pi(a|s)$

以上说来说去，其实是在说：策略影响了状态的访问分布。